**Environment and Data Initialization**

In [287]:
# Import the required libraries
import time
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy import text

In [288]:
# Read the dataset into the dataframe for the analysis

walmart_df = pd.read_csv("C:\\Users\\Admin\\Documents\\OneDrive\\Desktop\\Python\\Walmart Sales Analysis\\Datasets\\Walmart.csv")


**Data Transformation & Cleaning**

In [289]:
# Check the srtucture of the dataframe
print(walmart_df.info())

# Check the shape of the dataframe
print('Shape:', walmart_df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10051 entries, 0 to 10050
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      10051 non-null  int64  
 1   Branch          10051 non-null  object 
 2   City            10051 non-null  object 
 3   category        10051 non-null  object 
 4   unit_price      10020 non-null  object 
 5   quantity        10020 non-null  float64
 6   date            10051 non-null  object 
 7   time            10051 non-null  object 
 8   payment_method  10051 non-null  object 
 9   rating          10051 non-null  float64
 10  profit_margin   10051 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 863.9+ KB
None
Shape: (10051, 11)


In [290]:
# Check the statistical summary of the dataframe
print(walmart_df.describe())

print('\n')

# Check for quantity of missing values in the dataframe
print(walmart_df.isnull().sum())

print('\n')

# Check for duplicate rows in the dataframe
duplicate_rows = walmart_df.duplicated().sum()
print('Number of duplicate rows:', duplicate_rows)

         invoice_id      quantity        rating  profit_margin
count  10051.000000  10020.000000  10051.000000   10051.000000
mean    5025.741220      2.353493      5.825659       0.393791
std     2901.174372      1.602658      1.763991       0.090669
min        1.000000      1.000000      3.000000       0.180000
25%     2513.500000      1.000000      4.000000       0.330000
50%     5026.000000      2.000000      6.000000       0.330000
75%     7538.500000      3.000000      7.000000       0.480000
max    10000.000000     10.000000     10.000000       0.570000


invoice_id         0
Branch             0
City               0
category           0
unit_price        31
quantity          31
date               0
time               0
payment_method     0
rating             0
profit_margin      0
dtype: int64


Number of duplicate rows: 51


In [291]:
# Handling dupicated rows
# For our case, we will remove the duplicated rows
if duplicate_rows > 0:
    walmart_df.drop_duplicates(inplace=True)
    print('Duplicate rows removed.')

# Verify removal of duplicate rows
print('Number of duplicate rows after removal:', walmart_df.duplicated().sum())

Duplicate rows removed.
Number of duplicate rows after removal: 0


In [292]:
# Check the structure of the data
print(walmart_df.info())

print('\n')

# Check the shape of the data
print('Shape:', walmart_df.shape)

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      10000 non-null  int64  
 1   Branch          10000 non-null  object 
 2   City            10000 non-null  object 
 3   category        10000 non-null  object 
 4   unit_price      9969 non-null   object 
 5   quantity        9969 non-null   float64
 6   date            10000 non-null  object 
 7   time            10000 non-null  object 
 8   payment_method  10000 non-null  object 
 9   rating          10000 non-null  float64
 10  profit_margin   10000 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 937.5+ KB
None


Shape: (10000, 11)


In [293]:
# Dealing with missing values
# For our case, we will drop the rows with missing values
walmart_df.dropna(inplace=True)

# Verify removal of missing values
print('Missing values after removal:\n', walmart_df.isnull().sum())

print('\n')

# Final shape and structure of the cleaned dataframe
print(walmart_df.info())

print('\n')

print('Final shape of the dataframe:', walmart_df.shape)


Missing values after removal:
 invoice_id        0
Branch            0
City              0
category          0
unit_price        0
quantity          0
date              0
time              0
payment_method    0
rating            0
profit_margin     0
dtype: int64


<class 'pandas.core.frame.DataFrame'>
Index: 9969 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      9969 non-null   int64  
 1   Branch          9969 non-null   object 
 2   City            9969 non-null   object 
 3   category        9969 non-null   object 
 4   unit_price      9969 non-null   object 
 5   quantity        9969 non-null   float64
 6   date            9969 non-null   object 
 7   time            9969 non-null   object 
 8   payment_method  9969 non-null   object 
 9   rating          9969 non-null   float64
 10  profit_margin   9969 non-null   float64
dtypes: float64(3), int64(1), object(7)
memory 

In [294]:
# If we check the data again, we can see that the unit_price column has object data type which is not correct as it should be float.
print(walmart_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 9969 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      9969 non-null   int64  
 1   Branch          9969 non-null   object 
 2   City            9969 non-null   object 
 3   category        9969 non-null   object 
 4   unit_price      9969 non-null   object 
 5   quantity        9969 non-null   float64
 6   date            9969 non-null   object 
 7   time            9969 non-null   object 
 8   payment_method  9969 non-null   object 
 9   rating          9969 non-null   float64
 10  profit_margin   9969 non-null   float64
dtypes: float64(3), int64(1), object(7)
memory usage: 934.6+ KB
None


In [295]:
# Therefore, we need to convert the unit price column to float data type. 
# The presence of $ sign does not allow us to convert the column directly to float. Hence, we will first remove the $ sign from the column and then convert it to float.
walmart_df['unit_price'] = walmart_df['unit_price'].str.replace('$', '').astype(float)

# Verify the data type conversion
print(walmart_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 9969 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      9969 non-null   int64  
 1   Branch          9969 non-null   object 
 2   City            9969 non-null   object 
 3   category        9969 non-null   object 
 4   unit_price      9969 non-null   float64
 5   quantity        9969 non-null   float64
 6   date            9969 non-null   object 
 7   time            9969 non-null   object 
 8   payment_method  9969 non-null   object 
 9   rating          9969 non-null   float64
 10  profit_margin   9969 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 934.6+ KB
None


In [296]:
# We also need to convert the date and time columns to a date data type.
walmart_df['date'] = pd.to_datetime(walmart_df['date'])
walmart_df['time'] = pd.to_datetime(walmart_df['time'], format='%H:%M:%S').dt.time

# Verify the date data type conversion
print(walmart_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 9969 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   invoice_id      9969 non-null   int64         
 1   Branch          9969 non-null   object        
 2   City            9969 non-null   object        
 3   category        9969 non-null   object        
 4   unit_price      9969 non-null   float64       
 5   quantity        9969 non-null   float64       
 6   date            9969 non-null   datetime64[ns]
 7   time            9969 non-null   object        
 8   payment_method  9969 non-null   object        
 9   rating          9969 non-null   float64       
 10  profit_margin   9969 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(5)
memory usage: 934.6+ KB
None


C:\Users\Admin\AppData\Local\Temp\ipykernel_8588\2163649114.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  walmart_df['date'] = pd.to_datetime(walmart_df['date'])


In [297]:
# Check the first few rows of the cleaned dataframe
walmart_df.head()

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin
0,1,WALM003,San Antonio,Health and beauty,74.69,7.0,2019-05-01,13:08:00,Ewallet,9.1,0.48
1,2,WALM048,Harlingen,Electronic accessories,15.28,5.0,2019-08-03,10:29:00,Cash,9.6,0.48
2,3,WALM067,Haltom City,Home and lifestyle,46.33,7.0,2019-03-03,13:23:00,Credit card,7.4,0.33
3,4,WALM064,Bedford,Health and beauty,58.22,8.0,2019-01-27,20:33:00,Ewallet,8.4,0.33
4,5,WALM013,Irving,Sports and travel,86.31,7.0,2019-08-02,10:37:00,Ewallet,5.3,0.48


In [298]:
# Now that the unit_price column is in the correct format, we can proceed further with our transformations.
# We will create a new column, total_sales, which will be the product of quantity and unit_price columns.
walmart_df['total_sales'] = walmart_df['quantity'] * walmart_df['unit_price']

# We will also rename the column `Branch` to `branch` and `City` to `city` for consistency.
walmart_df.rename(columns={'Branch': 'branch', 'City': 'city'}, inplace=True)

In [299]:
# The dataframe is now cleaned and ready for further analysis.
walmart_df.head(10)

,invoice_id,branch,city,category,unit_price,quantity,date,time,payment_method,rating,profit_margin,total_sales
0,1,WALM003,San Antonio,Health and beauty,74.69,7.0,2019-05-01,13:08:00,Ewallet,9.1,0.48,522.83
1,2,WALM048,Harlingen,Electronic accessories,15.28,5.0,2019-08-03,10:29:00,Cash,9.6,0.48,76.40
2,3,WALM067,Haltom City,Home and lifestyle,46.33,7.0,2019-03-03,13:23:00,Credit card,7.4,0.33,324.31
3,4,WALM064,Bedford,Health and beauty,58.22,8.0,2019-01-27,20:33:00,Ewallet,8.4,0.33,465.76
4,5,WALM013,Irving,Sports and travel,86.31,7.0,2019-08-02,10:37:00,Ewallet,5.3,0.48,604.17
5,6,WALM026,Denton,Electronic accessories,85.39,7.0,2019-03-25,18:30:00,Ewallet,4.1,0.48,597.73
6,7,WALM088,Cleburne,Electronic accessories,68.84,6.0,2019-02-25,14:36:00,Ewallet,5.8,0.33,413.04
7,8,WALM100,Canyon,Home and lifestyle,73.56,10.0,2019-02-24,11:38:00,Ewallet,8.0,0.18,735.60
8,9,WALM066,Grapevine,Health and beauty,36.26,2.0,2019-10-01,17:15:00,Credit card,7.2,0.33,72.52
9,10,WALM065,Texas City,Food and beverages,54.84,3.0,2019-02-20,13:27:00,Credit card,5.9,0.33,164.52


**Connecting to SQL Server & Initializing Database**

In [300]:
# Now we will create a connection to the SQL Server, create a new database and store the cleaned dataframe into a new table in that database.
# Define the connection parameters
server = 'ANHADSLAPTOP\\SQLEXPRESS'
driver = 'ODBC Driver 17 for SQL Server'
Trusted_Connection = 'yes'
TrustServerCertificate = 'yes'

# Close existing connection if any
if 'sql_connection' in locals():
    sql_connection.close()

# Create new connection to master database with autocommit
connection_string = f'DRIVER={{{driver}}};SERVER={server};DATABASE=master;Trusted_Connection={Trusted_Connection};TrustServerCertificate={TrustServerCertificate}'
sql_connection = pyodbc.connect(connection_string, autocommit=True)
cursor = sql_connection.cursor()


In [301]:
# Define and execute the database creation query
query = """
IF DB_ID('walmart_db') IS NOT NULL
BEGIN
    ALTER DATABASE walmart_db SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
    DROP DATABASE walmart_db;
END;

CREATE DATABASE walmart_db;
"""

cursor.execute(query)

In [302]:
# Now let's connect to the newly created database
sql_connection.close()
connection_string = f'DRIVER={{{driver}}};SERVER={server};DATABASE=walmart_db;Trusted_Connection={Trusted_Connection};TrustServerCertificate={TrustServerCertificate}'
sql_connection = pyodbc.connect(connection_string, autocommit=True)
driver_encoded = 'ODBC+Driver+17+for+SQL+Server'

# Create a SQLAlchemy engine
engine = sqlalchemy.create_engine(
    f"mssql+pyodbc://@{server}/walmart_db?driver={driver_encoded}&trusted_connection=yes&TrustServerCertificate=yes"
)

try:
    # Store the cleaned dataframe into a new table in the database
    walmart_df.to_sql('walmart_sales', con=engine, if_exists='replace', index=False)
    print("DataFrame successfully stored in the database table 'walmart_sales'.")
except:
    print("An error occurred while storing the DataFrame in the database.")

DataFrame successfully stored in the database table 'walmart_sales'.


In [303]:
# Let's create a function that takes a SQL query as input and returns the result as a pandas dataframe.
def execute_query(query):
    start = time.time()
    with engine.connect() as conn:
        result = conn.execute(text(query))
        df_result = pd.DataFrame(result.fetchall(), columns=result.keys())
    print(f"Query executed in {time.time() - start:.3f} seconds.")
    return df_result

**Preliminary Analysis & Data Exploration**

In [304]:
# Test the data query 
query1 = \
"   SELECT TOP 5 *              \
    FROM dbo.walmart_sales      \
    WHERE total_sales > 500     "
execute_query(query1)

Query executed in 0.022 seconds.


,invoice_id,branch,city,category,unit_price,quantity,date,time,payment_method,rating,profit_margin,total_sales
0,1,WALM003,San Antonio,Health and beauty,74.69,7.0,2019-05-01,13:08:00,Ewallet,9.1,0.48,522.83
1,5,WALM013,Irving,Sports and travel,86.31,7.0,2019-08-02,10:37:00,Ewallet,5.3,0.48,604.17
2,6,WALM026,Denton,Electronic accessories,85.39,7.0,2019-03-25,18:30:00,Ewallet,4.1,0.48,597.73
3,8,WALM100,Canyon,Home and lifestyle,73.56,10.0,2019-02-24,11:38:00,Ewallet,8.0,0.18,735.60
4,15,WALM031,Lewisville,Health and beauty,71.38,10.0,2019-03-29,19:21:00,Cash,5.7,0.48,713.80


In [305]:
# Get the total number of records in the walmart_sales table
query2 = \
"   SELECT COUNT(*) AS total_records    \
    FROM dbo.walmart_sales               "
execute_query(query2)

Query executed in 0.006 seconds.


,total_records
0,9969


In [306]:
# Get the number of distinct stores
query3 = \
"   SELECT COUNT(DISTINCT branch) AS total_records               \
    FROM dbo.walmart_sales                                       "
execute_query(query3)

Query executed in 0.060 seconds.


,total_records
0,100


In [307]:
# Get the maximum and minimum sales overall
query4 = \
"   SELECT                                                        \
        MAX(total_sales) AS max_sale_amount,                      \
        MIN(total_sales) AS min_sale_amount                       \
    FROM dbo.walmart_sales                                        "
execute_query(query4)

Query executed in 0.008 seconds.


,max_sale_amount,min_sale_amount
0,993.0,10.17


In [ ]:
# Get payment method-wise total sales, total quantity sold, and count of payments
query5 = \
"   SELECT                                              \
        payment_method,                                 \
        COUNT(*) AS count_payments,                     \
        SUM(total_sales) AS tot_sales,                  \
        CAST(SUM(quantity) AS INT) AS tot_quantity      \
    FROM dbo.walmart_sales                              \
    GROUP BY payment_method                             "
execute_query(query5)

Query executed in 0.052 seconds.


,payment_method,count_payments,tot_sales,tot_quantity
0,Credit card,4256,488821.02,9567
1,Cash,1832,263589.29,4984
2,Ewallet,3881,457316.07,8932


**Analytics Using Advanced SQL Concepts**

In [ ]:
# Get 10 top-rated branches based on the average customer ratings on each category within the branch
query6 = \
"   WITH cat_rank AS                                                                            \
    (                                                                                           \
    SELECT                                                                                      \
        branch,                                                                                 \
        category,                                                                               \
        ROUND(AVG(rating), 2) AS avg_rating,                                                    \
        ROW_NUMBER() OVER(PARTITION BY branch ORDER BY AVG(rating) DESC) AS rank_category       \
    FROM dbo.walmart_sales                                                                      \
    GROUP BY                                                                                    \
        branch,                                                                                 \
        category                                                                                \
    )                                                                                           \
    SELECT TOP 10                                                                               \
        cr.branch,                                                                              \
        cr.category,                                                                            \
        cr.avg_rating                                                                           \
    FROM cat_rank cr                                                                            \
    WHERE rank_category = 1                                                                     \
    ORDER BY cr.avg_rating DESC                                                                 "
execute_query(query6)

Query executed in 0.113 seconds.


,branch,category,avg_rating
0,WALM034,Health and beauty,10.0
1,WALM060,Health and beauty,9.9
2,WALM086,Health and beauty,9.9
3,WALM098,Health and beauty,9.8
4,WALM027,Health and beauty,9.7
5,WALM067,Sports and travel,9.7
6,WALM068,Electronic accessories,9.7
7,WALM009,Sports and travel,9.6
8,WALM048,Electronic accessories,9.6
9,WALM073,Food and beverages,9.6


In [310]:
# Get the top-rated category for each branch based on average customer ratings
query7 = \
"   WITH cat_rank AS (                                            \
        SELECT                                                    \
            branch,                                               \
            category,                                             \
            ROUND(AVG(rating), 2) AS avg_rating,                  \
            ROW_NUMBER() OVER(PARTITION BY branch ORDER BY AVG(rating) DESC) AS rank_category \
        FROM walmart_sales                                        \
        GROUP BY branch, category                                 \
    )                                                             \
    SELECT                                                        \
        cr.branch, cr.category, cr.avg_rating                     \
    FROM cat_rank cr                                              \
    WHERE rank_category = 1                                       "
execute_query(query7)

Query executed in 0.068 seconds.


,branch,category,avg_rating
0,WALM001,Electronic accessories,7.45
1,WALM002,Food and beverages,8.25
2,WALM003,Sports and travel,7.50
3,WALM004,Food and beverages,9.30
4,WALM005,Health and beauty,8.37
...,...,...,...
95,WALM096,Sports and travel,9.60
96,WALM097,Food and beverages,7.67
97,WALM098,Health and beauty,9.80
98,WALM099,Electronic accessories,5.95


In [311]:
# Get the day with the highest number of transactions for each branch
query8 = \
"   WITH per_day_trans AS (                                       \
        SELECT                                                    \
            ws.branch,                                            \
            CAST(ws.date AS DATE) AS date_sale,                   \
            FORMAT(ws.date,'dddd') AS day_name,                   \
            COUNT(*) AS tot_transactions,                         \
            ROW_NUMBER() OVER(PARTITION BY branch ORDER BY COUNT(*) DESC) AS rank_trans \
        FROM dbo.walmart_sales ws                                 \
        GROUP BY ws.branch, ws.date                               \
    )                                                             \
    SELECT                                                        \
        pdt.branch, pdt.date_sale, pdt.day_name, pdt.tot_transactions \
    FROM per_day_trans pdt                                        \
    WHERE rank_trans = 1                                          "
execute_query(query8)

Query executed in 0.057 seconds.


,branch,date_sale,day_name,tot_transactions
0,WALM001,2019-03-22,Friday,2
1,WALM002,2023-12-10,Sunday,1
2,WALM003,2020-11-27,Friday,3
3,WALM004,2021-07-08,Thursday,2
4,WALM005,2023-08-30,Wednesday,3
...,...,...,...,...
95,WALM096,2022-12-26,Monday,2
96,WALM097,2022-08-29,Monday,2
97,WALM098,2022-12-18,Sunday,2
98,WALM099,2021-12-23,Thursday,3


In [312]:
# Get the city and category-wise maximum, minimum, and average customer ratings
query9 = \
"   SELECT                                                        \
        city,                                                     \
        category,                                                 \
        MAX(rating) AS max_rat,                                   \
        MIN(rating) AS min_rat,                                   \
        ROUND(AVG(rating), 1) AS avg_rat                          \
    FROM dbo.walmart_sales                                        \
    GROUP BY city, category                                       \
    ORDER BY city, category                                       "
execute_query(query9)


Query executed in 0.097 seconds.


,city,category,max_rat,min_rat,avg_rat
0,Abilene,Electronic accessories,8.8,7.1,8.0
1,Abilene,Fashion accessories,9.0,4.0,6.2
2,Abilene,Food and beverages,8.9,6.0,7.0
3,Abilene,Health and beauty,9.7,9.7,9.7
4,Abilene,Home and lifestyle,9.0,4.0,6.1
...,...,...,...,...,...
508,Weslaco,Fashion accessories,9.5,3.0,5.0
509,Weslaco,Food and beverages,9.8,7.1,8.7
510,Weslaco,Health and beauty,9.2,4.3,6.8
511,Weslaco,Home and lifestyle,9.2,3.0,5.2


In [313]:
# Get the category-wise total sales and total profit, ordered by total profit in descending order
query10 = \
"   SELECT                                                        \
        fsq.category,                                             \
        ROUND(SUM(fsq.total_sales), 2) AS tot_sales,              \
        ROUND(SUM(fsq.profit_of_all_sales), 2) AS tot_profit      \
    FROM (                                                        \
        SELECT                                                    \
            category,                                             \
            total_sales,                                          \
            total_sales * profit_margin AS profit_of_all_sales    \
        FROM dbo.walmart_sales                                    \
    ) AS fsq                                                      \
    GROUP BY fsq.category                                         \
    ORDER BY tot_profit DESC                                      "
execute_query(query10)

Query executed in 0.014 seconds.


,category,tot_sales,tot_profit
0,Fashion accessories,489480.90,192314.89
1,Home and lifestyle,489250.06,192213.64
2,Electronic accessories,78175.03,30772.49
3,Food and beverages,53471.28,21552.86
4,Sports and travel,52497.93,20613.81
5,Health and beauty,46851.18,18671.73


In [314]:
# Get the most popular payment method for each branch
query11 = \
"   SELECT                                                        \
        fsq.branch, fsq.payment_method                            \
    FROM (                                                        \
        SELECT                                                    \
            branch, payment_method,                               \
            COUNT(*) AS total_transactions,                       \
            ROW_NUMBER() OVER(PARTITION BY branch ORDER BY COUNT(*) DESC) AS transaction_rank \
        FROM dbo.walmart_sales                                    \
        GROUP BY branch, payment_method                           \
    ) AS fsq                                                      \
    WHERE fsq.transaction_rank = 1                                \
    ORDER BY fsq.branch                                           "
execute_query(query11)

Query executed in 0.057 seconds.


,branch,payment_method
0,WALM001,Ewallet
1,WALM002,Ewallet
2,WALM003,Credit card
3,WALM004,Ewallet
4,WALM005,Ewallet
...,...,...
95,WALM096,Ewallet
96,WALM097,Ewallet
97,WALM098,Ewallet
98,WALM099,Credit card


In [315]:
# Get the distribution of sales transactions across different times of the day
query12 = \
"   WITH categorized_sales AS (                                   \
        SELECT *,                                                 \
            CASE                                                  \
                WHEN time >= '00:00:00' AND time < '12:00:00' THEN 'Morning' \
                WHEN time < '17:00:00' THEN 'Afternoon'           \
                ELSE 'Night'                                      \
            END AS sale_time_category                             \
        FROM dbo.walmart_sales ws                                 \
    )                                                             \
    SELECT                                                        \
        cs.sale_time_category, COUNT(*) AS total_transactions     \
    FROM categorized_sales cs                                     \
    GROUP BY cs.sale_time_category                                \
    ORDER BY total_transactions DESC                              "
execute_query(query12)

Query executed in 0.022 seconds.


,sale_time_category,total_transactions
0,Night,4273
1,Afternoon,3609
2,Morning,2087


In [316]:
# Identify bottom 5 branches based on Year-over-Year (YoY) percentage change in total sales from 2022 to 2023
query13 = \
"   WITH yoy_analysis AS (                                        \
        SELECT                                                    \
            branch,                                               \
            YEAR(date) AS year_of_sale,                           \
            ROUND(SUM(total_sales), 2) AS tot_sales,              \
            LAG(ROUND(SUM(total_sales), 2)) OVER(PARTITION BY branch ORDER BY YEAR(date)) AS prev_tot_sales \
        FROM dbo.walmart_sales                                    \
        WHERE YEAR(date) IN (2022, 2023)                          \
        GROUP BY branch, YEAR(date)                               \
    )                                                             \
    SELECT TOP 5                                                  \
        ya.branch, ya.tot_sales, ya.prev_tot_sales,               \
        ROUND(((ya.tot_sales - ya.prev_tot_sales)/ya.prev_tot_sales) * 100, 2) AS perc_change \
    FROM yoy_analysis ya                                          \
    WHERE ya.prev_tot_sales IS NOT NULL                           \
    ORDER BY perc_change                                          "
execute_query(query13)

Query executed in 0.028 seconds.


,branch,tot_sales,prev_tot_sales,perc_change
0,WALM045,647.0,1731.0,-62.62
1,WALM047,1069.0,2581.0,-58.58
2,WALM098,1030.0,2446.0,-57.89
3,WALM033,931.0,2099.0,-55.65
4,WALM081,850.0,1723.0,-50.67
